Connect to db. See instructions in [Setup](./00_Setup.ipynb).


In [1]:
import os
import datajoint as dj
import numpy as np

# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
dj.config["enable_python_native_blobs"] = True
dj.config.load("dj_local_conf.json")  # load config for database connection info

%load_ext autoreload
%autoreload 2

import


In [2]:
import spyglass.common as sgc
import spyglass.spikesorting.v1 as sgs
import spyglass.data_import as sgi

[2024-02-19 16:54:30,495][INFO]: Connecting dhshin@lmf-db.cin.ucsf.edu:3306
[2024-02-19 16:54:30,532][INFO]: Connected dhshin@lmf-db.cin.ucsf.edu:3306
[16:54:31][WARNING] Spyglass: Please update position_tools to >= 0.1.0


insert LabMember and Session


In [3]:
nwb_file_name = "mediumnwb20230802.nwb"
nwb_file_name2 = "mediumnwb20230802_.nwb"

In [6]:
#sgc.LabMember.insert_from_nwbfile(nwb_file_name)

In [4]:
sgi.insert_sessions(nwb_file_name)

/home/dhshin/projects/continual-recording/spyglass/src/spyglass/data_import/insert_sessions.py:58: UserWarning: Cannot insert data from mediumnwb20230802.nwb: mediumnwb20230802_.nwb is already in Nwbfile table.
  warnings.warn(


In [5]:
sgc.Session()

nwb_file_name name of the NWB file,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
aj8020230628_.nwb,aj80,"University of California, San Francisco",Loren Frank Lab,1,linear track,2023-06-28 12:22:41,1970-01-01 00:00:00,rat locomotion analysis
aj8420230703_.nwb,AJ84,"University of California, San Francisco",Loren Frank Lab,1,linear track,2023-07-03 16:19:52,1970-01-01 00:00:00,rat locomotion analysis
aj8620230628_.nwb,AJ86,"University of California, San Francisco",Loren Frank Lab,1,linear track,2023-06-28 14:17:06,1970-01-01 00:00:00,rat locomotion analysis
arthur20220314_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220315,Hippocampus content feedback,2022-03-15 10:12:09,1970-01-01 00:00:00,Hippocampus content feedback
arthur20220315_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220315,Hippocampus content feedback,2022-03-15 10:12:09,1970-01-01 00:00:00,Hippocampus content feedback
arthur20220316_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220316,Hippocampus content feedback,2022-03-16 10:03:06,1970-01-01 00:00:00,Hippocampus content feedback
arthur20220317_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220317,Hippocampus content feedback,2022-03-17 10:18:59,1970-01-01 00:00:00,Hippocampus content feedback
arthur20220318_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220318,Hippocampus content feedback,2022-03-18 10:29:58,1970-01-01 00:00:00,Hippocampus content feedback
arthur20220319_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220319,Hippocampus content feedback,2022-03-19 11:54:02,1970-01-01 00:00:00,Hippocampus content feedback
arthur20220320_.nwb,arthur,"University of California, San Francisco",Loren Frank,arthur_20220320,Hippocampus content feedback,2022-03-20 10:21:08,1970-01-01 00:00:00,Hippocampus content feedback


insert SortGroup


In [10]:
#sgs.SortGroup.set_group_by_shank(nwb_file_name=nwb_file_name2)

insert SpikeSortingRecordingSelection. use `insert_selection` method. this automatically generates a unique recording id


In [6]:
key = {
    "nwb_file_name": nwb_file_name2,
    "sort_group_id": 0,
    "preproc_param_name": "default",
}

In [7]:
sgs.SpikeSortingRecordingSelection() & key

recording_id,nwb_file_name name of the NWB file,sort_group_id,interval_list_name descriptive name of this interval list,preproc_param_name,team_name
747f4eea-6df3-422b-941e-b5aaad7ec607,mediumnwb20230802_.nwb,0,02_r1,default,ms_stim


In [8]:
sgs.SpikeSortingRecordingSelection.insert_selection(key)

[16:54:33][WARNING] Spyglass: Similar row(s) already inserted.


[{'recording_id': UUID('747f4eea-6df3-422b-941e-b5aaad7ec607'),
  'nwb_file_name': 'mediumnwb20230802_.nwb',
  'sort_group_id': 0,
  'interval_list_name': '02_r1',
  'preproc_param_name': 'default',
  'team_name': 'ms_stim'}]

In [11]:
# Assuming 'key' is a dictionary with fields that you want to include in 'ssr_key'
ssr_key = {
    "recording_id": (sgs.SpikeSortingRecordingSelection() & key).fetch1('recording_id'),	
}

# Update ssr_key with all items from 'key'
ssr_key.update(key)

# Optionally, add or update additional fields in 'ssr_key'
# Uncomment and adjust the following lines according to your specific requirements
# ssr_key["n
ssr_key

{'recording_id': UUID('747f4eea-6df3-422b-941e-b5aaad7ec607'),
 'nwb_file_name': 'mediumnwb20230802_.nwb',
 'sort_group_id': 0,
 'preproc_param_name': 'default'}

preprocess recording (filtering and referencing)


In [12]:
#sgs.SpikeSortingRecording.populate()
sgs.SpikeSortingRecording.populate(ssr_key)

{'success_count': 0, 'error_list': []}

In [14]:
sgs.SpikeSortingRecording() & ssr_key

recording_id,analysis_file_name name of the file,object_id Object ID for the processed recording in NWB file
747f4eea-6df3-422b-941e-b5aaad7ec607,mediumnwb20230802_1UWNT3RC39.nwb,d18630b4-cb00-4446-b2b4-5b4a041804c8


In [15]:
key = (
    sgs.SpikeSortingRecordingSelection & key
).fetch1()

insert ArtifactDetectionSelection


In [16]:
sgs.ArtifactDetectionSelection.insert_selection(
    {"recording_id": key["recording_id"], "artifact_param_name": "default"}
)

[16:55:06][WARNING] Spyglass: Similar row(s) already inserted.


[{'artifact_id': UUID('9cf9e3cd-7115-4b59-a718-3633725d4738'),
  'recording_id': UUID('747f4eea-6df3-422b-941e-b5aaad7ec607'),
  'artifact_param_name': 'default'}]

detect artifact; note the output is stored in IntervalList


In [17]:
sgs.ArtifactDetection.populate()

{'success_count': 0, 'error_list': []}

In [20]:
sgs.ArtifactDetection()

artifact_id
0a6611b3-c593-4900-a715-66bb1396940e
1cd6ad2c-8ed6-447e-bbf8-d60e77df418c
27249d21-f04f-4738-b21f-4ad4a2bc58f7
2994ba9e-8fa9-4f87-8691-ccde704f5716
2cdba450-ca51-48ca-b90a-5fed93b079d4
2f89510f-f64a-4fb7-832b-3e4a0b8c41b9
3404cd5b-59b2-412d-93ab-5053796951b2
38945e52-fcd0-4005-829f-4439320f03b2
3d0f32bd-aea9-408d-904f-671024d93b78
3d782852-a56b-4a9d-89ca-be9e1a15c957


insert SpikeSortingSelection. again use `insert_selection` method. This example is with mountainsort4


In [21]:
key = {
    "recording_id": key["recording_id"],
    "sorter": "mountainsort4",
    "sorter_param_name": "franklab_tetrode_hippocampus_30KHz",
    "nwb_file_name": nwb_file_name2,
    "interval_list_name": str(
        (
            sgs.ArtifactDetectionSelection
            & {"recording_id": key["recording_id"]}
        ).fetch1("artifact_id")
    ),
}

# When using other sorter -- change the sorter name below (tested with mountainsort5, kilosort2_5,kilosort3
# when using mountainsort5, pip install 'mountainsort5'
# when using kilosort -- make sure to pip install 'cuda-python' and 'spython'
# For sorting with Kilosort, make sure to put the whole probe not a sliced individual shank.

sorter = 'kilosort2_5'
'''
key = {
    "recording_id": key["recording_id"],
    "sorter": sorter
    "sorter_param_name": "default",
    "nwb_file_name": nwb_file_name2,
    "interval_list_name": str(
        (
            sgs.ArtifactDetectionSelection
            & {"recording_id": key["recording_id"]}
        ).fetch1("artifact_id")
    ),
}
'''

'\nkey = {\n    "recording_id": key["recording_id"],\n    "sorter": sorter\n    "sorter_param_name": "default",\n    "nwb_file_name": nwb_file_name2,\n    "interval_list_name": str(\n        (\n            sgs.ArtifactDetectionSelection\n            & {"recording_id": key["recording_id"]}\n        ).fetch1("artifact_id")\n    ),\n}\n'

In [22]:
sgs.SpikeSortingSelection.insert_selection(key)

[16:55:39][INFO] Spyglass: Similar row(s) already inserted.


[{'sorting_id': UUID('874775be-df0f-4850-8f88-59ba1bbead89'),
  'recording_id': UUID('747f4eea-6df3-422b-941e-b5aaad7ec607'),
  'sorter': 'mountainsort4',
  'sorter_param_name': 'franklab_tetrode_hippocampus_30KHz',
  'nwb_file_name': 'mediumnwb20230802_.nwb',
  'interval_list_name': '9cf9e3cd-7115-4b59-a718-3633725d4738'}]

In [23]:
sgs.SpikeSortingSelection() & key

sorting_id,recording_id,sorter,sorter_param_name,nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list
874775be-df0f-4850-8f88-59ba1bbead89,747f4eea-6df3-422b-941e-b5aaad7ec607,mountainsort4,franklab_tetrode_hippocampus_30KHz,mediumnwb20230802_.nwb,9cf9e3cd-7115-4b59-a718-3633725d4738


run spike sorting


In [24]:
sss_pk = (sgs.SpikeSortingSelection & key).proj()

sgs.SpikeSorting.populate(sss_pk)

{'success_count': 0, 'error_list': []}

we have two main ways of curating spike sorting: by computing quality metrics and applying threshold; and manually applying curation labels. to do so, we first insert CurationV1. use `insert_curation` method.


In [ ]:
sgs.CurationV1.insert_curation(
    sorting_id=(
        sgs.SpikeSortingSelection & {"recording_id": key["recording_id"]}
    ).fetch1("sorting_id"),
    description="testing sort",
)

In [ ]:
sgs.CurationV1()

we will first do an automatic curation based on quality metrics


In [ ]:
key = {
    "sorting_id": (
        sgs.SpikeSortingSelection & {"recording_id": key["recording_id"]}
    ).fetch1("sorting_id"),
    "curation_id": 0,
    "waveform_param_name": "default_not_whitened",
    "metric_param_name": "franklab_default",
    "metric_curation_param_name": "default",
}

In [ ]:
sgs.MetricCurationSelection.insert_selection(key)

In [ ]:
sgs.MetricCurationSelection()

In [ ]:
sgs.MetricCuration.populate()

In [ ]:
sgs.MetricCuration()

to do another round of curation, fetch the relevant info and insert back into CurationV1 using `insert_curation`


In [ ]:
key = {
    "metric_curation_id": (
        sgs.MetricCurationSelection & {"sorting_id": key["sorting_id"]}
    ).fetch1("metric_curation_id")
}

In [ ]:
labels = sgs.MetricCuration.get_labels(key)

In [ ]:
merge_groups = sgs.MetricCuration.get_merge_groups(key)

In [ ]:
metrics = sgs.MetricCuration.get_metrics(key)

In [ ]:
sgs.CurationV1.insert_curation(
    sorting_id=(
        sgs.MetricCurationSelection
        & {"metric_curation_id": key["metric_curation_id"]}
    ).fetch1("sorting_id"),
    parent_curation_id=0,
    labels=labels,
    merge_groups=merge_groups,
    metrics=metrics,
    description="after metric curation",
)

In [ ]:
sgs.CurationV1()

next we will do manual curation. this is done with figurl. to incorporate info from other stages of processing (e.g. metrics) we have to store that with kachery cloud and get curation uri referring to it. it can be done with `generate_curation_uri`.


In [ ]:
curation_uri = sgs.FigURLCurationSelection.generate_curation_uri(
    {
        "sorting_id": (
            sgs.MetricCurationSelection
            & {"metric_curation_id": key["metric_curation_id"]}
        ).fetch1("sorting_id"),
        "curation_id": 1,
    }
)

In [ ]:
key = {
    "sorting_id": (
        sgs.MetricCurationSelection
        & {"metric_curation_id": key["metric_curation_id"]}
    ).fetch1("sorting_id"),
    "curation_id": 1,
    "curation_uri": curation_uri,
    "metrics_figurl": list(metrics.keys()),
}

In [ ]:
sgs.FigURLCurationSelection()

In [ ]:
sgs.FigURLCurationSelection.insert_selection(key)

In [ ]:
sgs.FigURLCurationSelection()

In [ ]:
sgs.FigURLCuration.populate()

In [ ]:
sgs.FigURLCuration()

or you can manually specify it if you already have a `curation.json`


In [ ]:
gh_curation_uri = (
    "gh://LorenFrankLab/sorting-curations/main/khl02007/test/curation.json"
)

key = {
    "sorting_id": key["sorting_id"],
    "curation_id": 1,
    "curation_uri": gh_curation_uri,
    "metrics_figurl": [],
}

In [ ]:
sgs.FigURLCurationSelection.insert_selection(key)

In [ ]:
sgs.FigURLCuration.populate()

In [ ]:
sgs.FigURLCuration()

once you apply manual curation (curation labels and merge groups) you can store them as nwb by inserting another row in CurationV1. And then you can do more rounds of curation if you want.


In [ ]:
labels = sgs.FigURLCuration.get_labels(gh_curation_uri)

In [ ]:
merge_groups = sgs.FigURLCuration.get_merge_groups(gh_curation_uri)

In [ ]:
sgs.CurationV1.insert_curation(
    sorting_id=key["sorting_id"],
    parent_curation_id=1,
    labels=labels,
    merge_groups=merge_groups,
    metrics=metrics,
    description="after figurl curation",
)

In [ ]:
sgs.CurationV1()

We now insert the curated spike sorting to a `Merge` table for feeding into downstream processing pipelines.


In [ ]:
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput

In [ ]:
SpikeSortingOutput()

In [ ]:
key

In [ ]:
SpikeSortingOutput.insert([key], part_name="CurationV1")

In [ ]:
SpikeSortingOutput.merge_view()

In [ ]:
SpikeSortingOutput.CurationV1()

In [ ]:
SpikeSortingOutput.CuratedSpikeSorting()